In [1]:
import requests
from requests.exceptions import ConnectionError
import lxml.html as lh
import pandas as pd
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [2]:
companies1 = pd.read_csv('List_of_companies.csv', encoding = "ISO-8859-1") #unmodified names
companies = pd.read_csv('List_of_companies_Full.csv', encoding = "ISO-8859-1") #will have modified, correct names

print(companies.dtypes)

companies['Security'] = companies['Security'].str.replace(u'\xa0', u' ')#this was messing with the classes
companies['Security'] = companies['Security'].str.replace(r' \(Class A\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r' \(Class B\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r' \(Class C\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r'\(', '')#need to remove parenthesis too for the url
companies['Security'] = companies['Security'].str.replace(r'\)', '')
companies['Security'] = companies['Security'].str.replace(r'\.com', '-com')#for amazon and salesforce
companies['Security'] = companies['Security'].str.replace(r'Corporation', 'corp')#corporation got shortened to corp
companies['Security'] = companies['Security'].str.replace(r'corporation', 'corp')
companies['Security'] = companies['Security'].str.replace(r'Company', 'co')#company got shortened to co
companies['Security'] = companies['Security'].str.replace(r'company', 'co')
companies['Security'] = companies['Security'].str.replace(r'&', 'and')#& is and for some reason
companies['Security'] = companies['Security'].str.replace(r' ', '-')#the url uses hyphens a lot
companies['Security'] = companies['Security'].str.replace(r'.', '')#don't want these here
companies['Security'] = companies['Security'].str.replace(r',', '')
companies['Security'] = companies['Security'].str.replace(r'!', '')
companies['Security'] = companies['Security'].str.replace(r'\'', '')

companies['Security'] = companies['Security'].str.lower()#lowercases everything
print(len(companies))
companies

Symbol                   object
Security                 object
SEC filings              object
GICS Sector              object
GICS Sub-Industry        object
Headquarters Location    object
Date first added         object
CIK                       int64
Founded                  object
dtype: object
500


,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3m-co,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",8/9/1976,66740,1902
1,ABT,abbott-laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800,1888
2,ABBV,abbvie-inc,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888)
3,ABMD,abiomed-inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094,1981
4,ACN,accenture-plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989
5,ATVI,activision-blizzard,reports,Communication Services,Interactive Home Entertainment,"Santa Monica, California",8/31/2015,718877,2008
6,ADBE,adobe-inc,reports,Information Technology,Application Software,"San Jose, California",5/5/1997,796343,1982
7,AMD,advanced-micro-devices-inc,reports,Information Technology,Semiconductors,"Santa Clara, California",3/20/2017,2488,1969
8,AAP,advance-auto-parts,reports,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",7/9/2015,1158449,1932
9,AES,aes-corp-the,reports,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",10/2/1998,874761,1981


In [3]:
#utilities
def getUrl(n, mod = ''):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + mod + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getIncUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-inc' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getCoUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-co' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getCorpUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-corp' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getPlcUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-plc' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getContent(page):
    doc=lh.fromstring(page.content)
    tr_elements=doc.xpath('//tr')
    return tr_elements

In [4]:
def failureCase(col, good, alt_tr_elements):
    for j in range(0,len(alt_tr_elements)):
        #T is our j'th row
        T=alt_tr_elements[j]

        #Only check rows of size 4 (with CEO Name, CEO Pay, Median Employee Pay,
        #and CEO Pay Ratio)
        if len(T)==4:
            #checks if the current row has the 4 maagic words
            table_Name = ['','','','']
            i=0
            for t in T:
                name = t.text_content()
                name = name.__str__()
                table_Name[i]=name
                i+=1

            #if it does, I'll take the data from the row right below it
            if (table_Name[0] == 'CEO Name' and table_Name[1] == 'CEO Pay' and table_Name[2] == 'Median Employee Pay' and table_Name[3] == 'CEO Pay Ratio'):
                T=alt_tr_elements[j+1]
                #i is the index of our column
                i=0

                #this is a temporary storage of the content of each line of the csv
                content = []

                #Iterate through each element of the row
                for t in T.iterchildren():
                    #this part if for cleaning the cell contents
                    data=t.text_content() #gets the content of cell
                    data = data.__str__() #turns content to string
                    data = data.replace(r'$', '') #takes out dollar sign for number conversion
                    data = data.replace(r',', '') #takes out comma for number conversion
                    data = data.strip() #clears our \r and \n and spaces, cleans up the content string
                    data = data.replace(r'CEO Pay', '') #cleans up the content string
                    data = data.replace(r'Median Employee Pay', '') #cleans up the content string
                    data = data.replace(r'CEO Pay Ratio', '') #cleans up the content string
                    data = data.replace(r'Ratio', '') #cleans up the content string
                    data = data.replace(r':1', '') #takes out :1 in the ratio for number conversion
                    data = data.strip() #clears our \r and \n and spaces again to finish cleaning up

                    content.append(data)

                    #Check if row is empty
                    if i>0:
                    #Convert any numerical value to integers
                        try:
                            data=int(data)
                        except:
                            pass
                    #Append the data to col
                    col[i + 1][1].append(data)
                    #Increment i for the next column
                    i+=1
                #col[0][1].append(currName1)
                return True

        return False

In [5]:
#Makes a list of to make the graph later on
col=[('Security', [])]

In [9]:
#goes through all the webpages
for n in range(201, len(companies)):
    currName1 = companies.at[n, 'Security']
    add = currName1
    #getting url
    url='https://www1.salary.com/' + currName1 + '-Executive-Salaries.html'
    page=requests.get(url)
    #getting all the tables
    tr_elements=getContent(page)
    #indicates that we failed to find the magics on this particular page
    fail = True
    for j in range(0,len(tr_elements)):
        #T is our j'th row
        T=tr_elements[j]

        #Only check rows of size 4 (with CEO Name, CEO Pay, Median Employee Pay,
        #and CEO Pay Ratio)
        if len(T)==4:
            #checks if the current row has the 4 maagic words
            table_Name = ['','','','']
            i=0
            for t in T:
                name = t.text_content()
                name = name.__str__()
                table_Name[i]=name
                i+=1

            #if it does, I'll take the data from the row right below it
            if (table_Name[0] == 'CEO Name' and table_Name[1] == 'CEO Pay' and table_Name[2] == 'Median Employee Pay' and table_Name[3] == 'CEO Pay Ratio'):
                #print(currName1 + ' good')
                #setting fail to false
                fail = False
                add = currName1
                break
    
    if fail:
        #print(currName1)
        #print(currName)
        #print(url)
        #put these into the get url
        alts = ['-inc','-co','-corp','-plc','-new']
        #good indicated whether or not the new url works
        good = False
        for i in range(len(alts)):
            currName = currName1 + alts[i]
            url='https://www1.salary.com/' + currName + '-Executive-Salaries.html'
            page=requests.get(url)
            alt_tr_elements=getContent(page)
            for j in range(0,len(alt_tr_elements)):
                #T is our j'th row
                T=alt_tr_elements[j]

                #Only check rows of size 4 (with CEO Name, CEO Pay, Median Employee Pay,
                #and CEO Pay Ratio)
                if len(T)==4:
                    #checks if the current row has the 4 maagic words
                    table_Name = ['','','','']
                    i=0
                    for t in T:
                        name = t.text_content()
                        name = name.__str__()
                        table_Name[i]=name
                        i+=1

                    #if it does, I'll take the data from the row right below it
                    if (table_Name[0] == 'CEO Name' and table_Name[1] == 'CEO Pay' and table_Name[2] == 'Median Employee Pay' and table_Name[3] == 'CEO Pay Ratio'):
                        #print(currName1 + ' good')
                        T=alt_tr_elements[j+1]
                        good = True
                        
            if good:
                add = currName
                break
    col[0][1].append(add)
    print(n, add)
    

201 franklin-resources-inc
202 freeport-mcmoran-inc
203 gap-inc
204 garmin-ltd
205 gartner-inc
206 general-dynamics-corp
207 general-electric-co
208 general-mills-inc
209 general-motors-co
210 genuine-parts-co
211 gilead-sciences-inc
212 torchmark-corp
213 global-payments-inc
214 goldman-sachs-group-inc
215 grainger-w-w-inc
216 halliburton-co
217 hanesbrands-inc
218 hartford-financial-services
219 hasbro-inc
220 hca-healthcare-inc
221 hcp-inc
222 henry-schein-inc
223 hershey-co
224 hess-corp
225 hewlett-packard-enterprise-co
226 hilton-worldwide-holdings-inc
227 hollyfrontier-corp
228 hologic-inc
229 home-depot-inc
230 honeywell-international-inc
231 hormel-foods-corp
232 host-hotels-and-resorts-inc
233 arconic-inc
234 hp-inc
235 humana-inc
236 huntington-bancshares
237 huntington-ingalls-ind-inc
238 idex-corp
239 idexx-labs-inc
240 ihs-markit-ltd
241 illinois-tool-works
242 illumina-inc
243 incyte-corp
244 ingersoll-rand-plc
245 intel-corp
246 intercontinental-exchange-inc
247 intl-bu

In [10]:
col

[('Security',
  ['3m-co',
   'abbott-laboratories',
   'abbvie-inc',
   'abiomed-inc',
   'accenture-plc',
   'activision-blizzard-inc',
   'adobe-inc',
   'advanced-micro-devices-inc',
   'advance-auto-parts-inc',
   'aes-corp-the',
   'aflac-inc',
   'agilent-technologies-inc',
   'air-products-and-chemicals-inc',
   'akamai-technologies-inc',
   'alaska-air-group-inc',
   'albemarle-corp',
   'alexandria-r-e-equities-inc',
   'alexion-pharmaceuticals-inc',
   'align-technology-inc',
   'allegion-plc',
   'alliant-energy-corp',
   'allstate-corp',
   'alphabet-inc',
   'altria-group-inc',
   'amazon-com-inc',
   'amcor-plc',
   'ameren-corp',
   'american-airlines-group-inc',
   'american-electric-power-co',
   'american-express-co',
   'american-international-group',
   'american-tower-corp-ma',
   'american-water-works-co-inc',
   'ameriprise-financial-inc',
   'amerisourcebergen-corp',
   'ametek-inc',
   'amgen-inc',
   'amphenol-corp',
   'analog-devices',
   'ansys-inc',
   'an

In [16]:
len(col[0][1])

500

In [17]:
#making the dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [18]:
#displaying the dataframe
df

,Security
0,3m-co
1,abbott-laboratories
2,abbvie-inc
3,abiomed-inc
4,accenture-plc
5,activision-blizzard-inc
6,adobe-inc
7,advanced-micro-devices-inc
8,advance-auto-parts-inc
9,aes-corp-the


In [ ]:
"""int = 0
for i in range(len(df)):
    if (df.at[i, 'Median Employee Pay'] > -1):
        int += 1
print(int)
print(len(df)-int)"""

int = 0
for i in range(len(df)):
    if (df.at[i, 'Median Employee Pay'] == -1):
        int += 1
        print(i)
        print(df.at[i, 'Company Name'])
int

In [ ]:
#verifying correctness

int = 0

for i in range(len(companies)):
    if (df_ratio.at[i, 'GICS Sector'] != companies1.at[i, 'GICS\xa0Sector'] or df_ratio.at[i, 'GICS Sub-Industry'] != companies1.at[i, 'GICS Sub-Industry']):
        int += 1
        print(df.at[i, 'Company Name'] + " is not " + companies1.at[i, 'Security'])
        print(i)
int

In [19]:
df.to_csv('new_quick_list.csv',index=False)

In [ ]:
#df_ratio.to_csv('official_ratio_with_industries.csv',index=False)